Import Libraries

In [1]:
import base64
import numpy as np
import pika
import threading
import requests
import face_recognition

Set Connection

In [2]:
host = '192.168.56.102' #La mia macchina virtuale
#host = '192.168.1.172'
exchange_name = 'camera'
routing_key_name = 'security'

connection = pika.BlockingConnection(pika.ConnectionParameters(host=host))
channel = connection.channel()

In [3]:
channel.exchange_declare(exchange=exchange_name, exchange_type='topic')

result = channel.queue_declare(queue="security office", exclusive=True)
queue_name = result.method.queue
channel.queue_bind(exchange = exchange_name,
                   queue = queue_name,
                   routing_key = routing_key_name)

<METHOD(['channel_number=1', 'frame_type=1', 'method=<Queue.BindOk>'])>

Set Actions

In [4]:
def detection(image):
    face_landmarks_list = face_recognition.face_landmarks(image)
    count=0
    for face_landmarks in face_landmarks_list:
        count += 1
    r = requests.post("http://192.168.56.102:32829", data=str(count))
    #print('Number of faces detected: ', count, ', ifttt sent')
    #send request to nuclio where it triggers the ifttt with the number of faces detected

In [5]:
def callback(ch, method, properties, body):        
        #Decode the frame
        decodedString = base64.b64decode(body)
        decodedImg = np.frombuffer(decodedString, dtype=np.uint8)
        decodedImg = decodedImg.reshape(480, 640, 3)
        
        #Start face detection
        detection_thread = threading.Thread(target=detection, args=(decodedImg,))
        detection_thread.start()


Consuming

In [ ]:
channel.basic_consume(callback,
                      queue=queue_name,
                      no_ack=True)

print('Waiting For Messages')
channel.start_consuming()

Waiting For Messages
